# Analisador Léxico

## L = {w in {a,b}* | w possui número impar de 'a's
<br>

<img src='img/AFD4.png'>

In [85]:
class ErroLexicoRxception(Exception):
    pass


class Analisador_Lexico:
    def __init__(self, buffer):
        self.buffer = buffer
        
    def __gerar_erro(self, erro):
        raise ErroLexicoRxception(f'Erro léxico: {erro}')

    
    def executar(self):
        i = 0
        estado = 0
        while i < len(self.buffer):
            simbolo = self.buffer[i]
            # Estado 0
            if estado ==0:
                
                if simbolo == 'a':
                    estado = 1
                    print(estado)
                elif simbolo =='b':
                    estado = 0
                    print(estado)
                else:
                    self.__gerar_erro('simbolo incorreto')
            # Estado 1      
            elif estado ==1:
                if simbolo == 'a':
                    estado = 0
                    print(estado)
                elif simbolo =='b':
                    estado = 1
                    print(estado)
                else:
                    self.__gerar_erro('simbolo incorreto')
            i+=1
        F = 1
        return F == estado
    

In [87]:
w = input('Digite a cadeia:')
lexico = Analisador_Lexico(w)
try:
    print(lexico.executar())
except ErroLexicoRxception as e:
    print(e)
    
    

Digite a cadeia:aaba
1
0
0
1
True


# Mini Analisador Léxico


<img src='img/ana1.png'>

<p>Autonomo para leitura de digitos e letras<p>
<img src='img/AFD5.png'>
    
<p>Autonomo para leitura números inteiros e decimais<p>
<img src='img/AFD6.png'>
    
<p>Autonomo para leitura números de operadores relacionais<p>
<img src='img/AFD7.png'>

In [132]:
from typing import NamedTuple, Union

class ErroLexicoRxception(Exception):
    pass

#tolkens da linguagem (tipo do tolken)
ERRO = 0
IDENTIFICADOR = 1
NUM_INT = 2
NUM_REAL = 3
EOS = 4 #fim da cadeia

tolken_msg = ['ERRO', 'IDENTIF', 'NUM_INT', 'NUM_REAL', 'EOS']

class Tolken(NamedTuple):
    tipo: int               #[ 0 - erro, 1 - indentificador, 2 - num_int, 3 - num_real, 4 - eos]     
    lexema: str
    valor: Union[int, float] # pode guardar um valor inteiro ou flutuante
    linha: int

class Analisador_Lexico:
    def __init__(self, buffer):
        self.nlinha = 1
        self.buffer = buffer + '\0' #indica o fim da string
        self.i = 0
        
    def proximo_char(self):
        c = self.buffer[self.i]
        self.i += 1
        return c
    
    def retrac_char(self):
        self.i -= 1
        
    def tratar_numero(self, c):
        lexema = c
        estado = 1

        c = self.proximo_char()
        while True:
            # estado 1
            if estado == 1:
                if  c.isdigit():
                    lexema = lexema + c
                    estado =1
                    c = self.proximo_char()
                elif c == '.':
                    lexema = lexema + c
                    estado = 3
                    c = self.proximo_char()
                else:
                    estado = 2
            
            # estado 2
            elif estado ==2:
                self.retrac_char()
                return Tolken(NUM_INT, lexema,int(lexema), self.nlinha)
            
            # estado 3
            elif estado ==3:
                if c.isdigit():
                    estado= 4
                else:
                    return Tolken(ERRO, '', 0, self.nlinha)
            
            # estado 4
            elif estado == 4:
                if c.isdigit():
                    lexema = lexema + c
                    estado = 4
                    c = self.proximo_char()
                else:
                    estado = 5
                
            # estado 5
            elif estado == 5:
                self.retrac_char()
                return Tolken(NUM_REAL, lexema, float(lexema), self.nlinha)


        return Tolken(NUM_INT, '', 0, self.nlinha)
    
    def tratar_identificador(self, c):
        lexema = c
        c = self.proximo_char()
        estado = 1

        while True:
            if estado ==1:
                if c.isdigit() or c.isalpha():
                    lexema = lexema + c
                    estado = 1
                    c = self.proximo_char()
                else:
                    estado = 2
            elif estado ==2:
                self.retrac_char()
                return Tolken(IDENTIFICADOR, lexema, 0, self.nlinha)
    
    def gerar_erro(self, erro):
        raise ErroLexicoRxception(f'Erro léxico: {erro}')
    
    def proximo_tolken(self):
        tolken = Tolken(ERRO, '', 0, self.nlinha)
        c = self.proximo_char()
        
        # Trata os delimitadores
        while c in [' ', '\n','\0']:
            
            if c == '\n':
                self.nlinha +=1
            
            if c =='\x00':
                return Tolken(EOS, '', 0, self.nlinha)
            
            c = self.proximo_char()
            #Tratar os outros Tolkens aqui
        if c.isdigit():
            return self.tratar_numero(c)
            
        if c.isalpha():
            return self.tratar_identificador(c)
            
        return tolken
        
def ler_arquivo():
    return '''111 1.4 34234 34234.4 dsfsdf sdfsdf
    asdasd
    asdas
    aaaa
    bbbb
    00
    0.0''' # implementar


def main():
    buffer = ler_arquivo()     # ler de um arquivo de entrada
    lexico = Analisador_Lexico(buffer)
    atomo = lexico.proximo_tolken()

    while (atomo.tipo != EOS and atomo.tipo != ERRO):
        print("Linha: {}".format(atomo.linha), end='')
        print(" - atomo: {} ".format(tolken_msg[atomo.tipo]), end='')
        print("\t\tlexema: {}".format(atomo.lexema), end='')
        print("\t\tvalor: {}\n".format(atomo.valor), end='')
        #print("\t\toperador: {}".format(operador_msg[atomo.operador]))
        atomo = lexico.proximo_tolken()

    print("Linha: {}".format(atomo.linha), end='')
    print(" - atomo: {} ".format(tolken_msg[atomo.tipo]), end='')
    print("\t\tlexema: {}".format(atomo.lexema), end='')
    print("\t\tvalor: {}\n".format(atomo.valor), end='')
    #print("\t\toperador: {}".format(operador_msg[atomo.operador]))

main()

Linha: 1 - atomo: NUM_INT 		lexema: 111		valor: 111
Linha: 1 - atomo: NUM_REAL 		lexema: 1.4		valor: 1.4
Linha: 1 - atomo: NUM_INT 		lexema: 34234		valor: 34234
Linha: 1 - atomo: NUM_REAL 		lexema: 34234.4		valor: 34234.4
Linha: 1 - atomo: IDENTIF 		lexema: dsfsdf		valor: 0
Linha: 1 - atomo: IDENTIF 		lexema: sdfsdf		valor: 0
Linha: 2 - atomo: IDENTIF 		lexema: asdasd		valor: 0
Linha: 3 - atomo: IDENTIF 		lexema: asdas		valor: 0
Linha: 4 - atomo: IDENTIF 		lexema: aaaa		valor: 0
Linha: 5 - atomo: IDENTIF 		lexema: bbbb		valor: 0
Linha: 6 - atomo: NUM_INT 		lexema: 00		valor: 0
Linha: 7 - atomo: NUM_REAL 		lexema: 0.0		valor: 0.0
Linha: 7 - atomo: EOS 		lexema: 		valor: 0


# Analisador Sintativo

$S ::= AS|BA$

$A ::= aB|d$

$B ::= bA|d$

$C ::= c$

$First(S) = First(AS) \cup First(BA)$

In [4]:
atomo_EOS = 0
atomo_a = 1
atomo_b = 2
atomo_c = 3
atomo_d = 4

#variaveis globais
i = 0
buf = ''
lookahead = 0

def proximo_char():
    global i
    if i < len(buf):
        c = buf[i]
        i += 1
        return c
    else:
        return '\0' # fim da cadeia
    
    
def proximo_atomo(): #analisador lexico
    c = proximo_char()
    if c == '\0':
        return atomo_EOS
    elif c == 'a':
        return atomo_a
    elif c == 'b':
        return atomo_b
    elif c == 'c':
        return atomo_c
    elif c == 'd':
        return atomo_d

def erro():
    print('erro sintatico')
        
def consome(atomo):
    global lookahead
    if (atomo == lookahead):
        lookahead = proximo_atomo()
    else:
        erro()
        
def sintatico():
    global lookahead
    lookahead = proximo_atomo()
    S() # gera arvore gramatical
    consome(atomo_EOS)

def A():
    if lookahead == atomo_a:
        consome(atomo_a)
        B()
    elif lookahead == atomo_c:
        C()    

def B():
    if lookahead == atomo_b:
        consome(atomo_b)
        A()
    elif lookahead == atomo_d:
        consome(atomo_d)

def C():
    consome(atomo_c)

def S():
    #first de S
    if lookahead == atomo_a or lookahead == atomo_c:
        A()
        S()
    elif lookahead == atomo_b or lookahead == atomo_d:
        B()
        A()

def main():
    global buf
    buf = input()
    sintatico()
    
main()  


aaaaaa


$E ::= E + T | T$

$T ::= T * F | F$

$F ::= a | b | (E)$

remoção da recursão esquerda (notação de Wirth estendida)

$E ::= T \{+T\}$

$T ::= F \{* F\}$

$F ::= a | b | (E)$

$FIRST(E) = \{a, b, (\}$

$FIRST(T) = \{a, b, (\}$

$FIRST(F) = \{ a, b, (\}$



In [20]:
atomo_EOS = 0
atomo_a = 1
atomo_b = 2
atomo_SOMA = 3
atomo_MULT = 4
atomo_ABREPAR = 5
atomo_FECHAPAR = 6

#variaveis globais
i = 0
buf = ''
lookahead = 0

def proximo_char():
    global i
    if i < len(buf):
        c = buf[i]
        i += 1
        return c
    else:
        return '\0' # fim da cadeia
    
def proximo_atomo(): # Analisador lexico
    c = proximo_char()
    if c == '\0':
        return atomo_EOS
    elif c == 'a':
        return atomo_a
    elif c == 'b':
        return atomo_b
    elif c == '+':
        return atomo_SOMA
    elif c == '*':
        return atomo_MULT
    elif c == '(':
        return atomo_ABREPAR
    elif c == ')':
        return atomo_FECHAPAR
    
def consome(atomo):
    global lookahead
    if (atomo == lookahead):
        lookahead = proximo_atomo()
    else:
        erro()
    
def E():
    T()
    while lookahead == atomo_SOMA:
        consome(atomo_SOMA)
        T()
        
def T():
    F()
    while lookahead == atomo_MULT:
        consome(atomo_MULT)
        F()
        
def F():
    if lookahead == atomo_a:
        consome(atomo_a)
    elif lookahead == atomo_b:
        consome(atomo_b)
    elif lookahead == atomo_ABREPAR:
        consome(atomo_ABREPAR)
        E()
        consome(atomo_FECHAPAR)
    
def sintatico():
    global lookahead
    lookahead = proximo_atomo()
    E()
    consome(atomo_EOS)
    print('fim da analise sintatica')
    
def main():
    global buf
    buf = input()
    sintatico()

main()

a+(a*(a+b))*b+(a*(a+a))
fim da analise sintatica
